In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from glob import glob
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
import json, folium
import numpy as np
from folium.features import DivIcon
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_list = []
for file in glob('C:/Users/kos/Downloads/지역*.xls'):
    gs = pd.read_excel(file, header=2)
    df_list.append(gs)

df = pd.concat(df_list)
df.shape

(464, 10)

In [3]:
df = df[['상호','주소','휘발유']]
df.head()

,상호,주소,휘발유
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1983
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1995
3,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),2015
4,대성석유(주)길동주유소,서울 강동구 천호대로 1168,2025


In [4]:
df.rename(columns={'휘발유':'가격'}, inplace=True)
df.tail()

,상호,주소,가격
31,(주)소모에너지 쎈트럴주유소,서울 강남구 삼성로 335,2455
32,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),2548
33,경원주유소,서울 강남구 도산대로 428 (청담동),-
34,동우주유소,서울특별시 강남구 봉은사로 311 (논현동),-
35,삼성주유소,서울 강남구 삼성로 521 (삼성동),-


In [5]:
df['구'] = df.주소.str.split().str[1]

In [6]:
df.구.unique()


array(['강동구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '강북구', '용산구', '은평구', '종로구', '중구', '중랑구', '강서구',
       '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '강남구'], dtype=object)

In [7]:
df = df[df.가격 != '-']
df.가격 = df.가격.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 0 to 32
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상호      456 non-null    object
 1   주소      456 non-null    object
 2   가격      456 non-null    int32 
 3   구       456 non-null    object
dtypes: int32(1), object(3)
memory usage: 16.0+ KB


In [9]:
df.dropna(subset=['가격'], inplace=True)
df.shape

(456, 4)

In [10]:
df.tail()

,상호,주소,가격,구
28,대청주유소,서울 강남구 개포로 654 (일원동),2336,강남구
29,갤러리아주유소,서울 강남구 압구정로 426,2440,강남구
30,SK논현주유소,서울 강남구 논현로 747 (논현동),2445,강남구
31,(주)소모에너지 쎈트럴주유소,서울 강남구 삼성로 335,2455,강남구
32,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),2548,강남구


In [11]:
lists_min = df.groupby('구')['가격'].agg(['min']).sort_values(by='min', ascending=True)
lists_max = df.groupby('구')['가격'].agg(['max']).sort_values(by='max', ascending=True)
lists_max

,max
구,
금천구,1970
은평구,2025
도봉구,2037
강북구,2039
광진구,2048
서대문구,2059
중랑구,2069
관악구,2087
성북구,2099


In [12]:
lists_max = lists_max.reset_index()
lists_min = lists_min.reset_index()
lists_min

,구,min
0,성동구,1898
1,강서구,1902
2,양천구,1902
3,서초구,1904
4,구로구,1919
5,강북구,1934
6,성북구,1935
7,금천구,1935
8,노원구,1935
9,동대문구,1935


In [13]:
df2 = pd.merge(df,lists_max)
df2 = pd.merge(df2, lists_min)


In [15]:
df2.head()

,상호,주소,가격,구,max,min
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,강동구,2298,1969
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1983,강동구,2298,1969
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1995,강동구,2298,1969
3,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),2015,강동구,2298,1969
4,대성석유(주)길동주유소,서울 강동구 천호대로 1168,2025,강동구,2298,1969


In [17]:
df2['test'] = (df2['가격'] == df2['max'] )
df2.head(3)

,상호,주소,가격,구,max,min,test
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,강동구,2298,1969,False
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1983,강동구,2298,1969,False
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1995,강동구,2298,1969,False


In [18]:
df_max = df2[df2['test'] == True]
df_max

,상호,주소,가격,구,max,min,test
13,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,강동구,2298,1969,True
32,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,동대문구,2190,1935,True
42,살피재주유소,서울 동작구 상도로 334,2135,동작구,2135,1964,True
54,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,마포구,2298,1968,True
69,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,서대문구,2059,1960,True
101,선우상사Q엔느,서울 서초구 방배로 66 (방배동),2497,서초구,2497,1904,True
118,동일주유소,서울 성동구 광나루로 254 (성수동2가),2328,성동구,2328,1898,True
142,㈜명연에너지 효성주유소,서울 성북구 화랑로 27,2099,성북구,2099,1935,True
174,대신석유(주)진양주유소,서울 송파구 올림픽로 499 (풍납동),2178,송파구,2178,1965,True
199,평화주유소,서울 양천구 안양천로 657 (신정동),2197,양천구,2197,1902,True


In [19]:
df_max = df_max.drop(columns='test')
df_max

,상호,주소,가격,구,max,min
13,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,강동구,2298,1969
32,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,동대문구,2190,1935
42,살피재주유소,서울 동작구 상도로 334,2135,동작구,2135,1964
54,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,마포구,2298,1968
69,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,서대문구,2059,1960
101,선우상사Q엔느,서울 서초구 방배로 66 (방배동),2497,서초구,2497,1904
118,동일주유소,서울 성동구 광나루로 254 (성수동2가),2328,성동구,2328,1898
142,㈜명연에너지 효성주유소,서울 성북구 화랑로 27,2099,성북구,2099,1935
174,대신석유(주)진양주유소,서울 송파구 올림픽로 499 (풍납동),2178,송파구,2178,1965
199,평화주유소,서울 양천구 안양천로 657 (신정동),2197,양천구,2197,1902


In [21]:
df2['test2'] = (df2['가격'] == df2['min'] )
df2.head()

,상호,주소,가격,구,max,min,test,test2
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,강동구,2298,1969,False,True
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1983,강동구,2298,1969,False,False
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1995,강동구,2298,1969,False,False
3,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),2015,강동구,2298,1969,False,False
4,대성석유(주)길동주유소,서울 강동구 천호대로 1168,2025,강동구,2298,1969,False,False


In [25]:
df_min = df2[df2['test2'] == True]
df_min

,상호,주소,가격,구,max,min,test,test2
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,강동구,2298,1969,False,True
14,대성산업㈜청량리주유소,서울 동대문구 왕산로 289 (청량리동),1935,동대문구,2190,1935,False,True
15,대성산업(주)동마장주유소,서울 동대문구 고산자로 405 (용두동),1935,동대문구,2190,1935,False,True
16,풍한주유소,서울 동대문구 안암로 168,1935,동대문구,2190,1935,False,True
17,동서울고속주유소,서울 동대문구 한천로 414,1935,동대문구,2190,1935,False,True
33,창원이엔지주유소,서울 동작구 사당로 59 (상도동),1964,동작구,2135,1964,False,True
43,청원주유소,서울 마포구 월드컵북로 113 (성산동),1968,마포구,2298,1968,False,True
44,성산대교셀프주유소,서울 마포구 성산로 144,1968,마포구,2298,1968,False,True
55,(주)창원이엔지 홍제지점,서울 서대문구 통일로 455 (홍제동),1960,서대문구,2059,1960,False,True
56,현대오일뱅크(주)직영 연세주유소,서울 서대문구 연희로 184,1960,서대문구,2059,1960,False,True


In [26]:
df_min = df_min.drop(columns='test')
df_min = df_min.drop(columns='test2')
df_min

,상호,주소,가격,구,max,min
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,강동구,2298,1969
14,대성산업㈜청량리주유소,서울 동대문구 왕산로 289 (청량리동),1935,동대문구,2190,1935
15,대성산업(주)동마장주유소,서울 동대문구 고산자로 405 (용두동),1935,동대문구,2190,1935
16,풍한주유소,서울 동대문구 안암로 168,1935,동대문구,2190,1935
17,동서울고속주유소,서울 동대문구 한천로 414,1935,동대문구,2190,1935
33,창원이엔지주유소,서울 동작구 사당로 59 (상도동),1964,동작구,2135,1964
43,청원주유소,서울 마포구 월드컵북로 113 (성산동),1968,마포구,2298,1968
44,성산대교셀프주유소,서울 마포구 성산로 144,1968,마포구,2298,1968
55,(주)창원이엔지 홍제지점,서울 서대문구 통일로 455 (홍제동),1960,서대문구,2059,1960
56,현대오일뱅크(주)직영 연세주유소,서울 서대문구 연희로 184,1960,서대문구,2059,1960


In [27]:
df_max

,상호,주소,가격,구,max,min
13,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,강동구,2298,1969
32,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,동대문구,2190,1935
42,살피재주유소,서울 동작구 상도로 334,2135,동작구,2135,1964
54,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,마포구,2298,1968
69,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,서대문구,2059,1960
101,선우상사Q엔느,서울 서초구 방배로 66 (방배동),2497,서초구,2497,1904
118,동일주유소,서울 성동구 광나루로 254 (성수동2가),2328,성동구,2328,1898
142,㈜명연에너지 효성주유소,서울 성북구 화랑로 27,2099,성북구,2099,1935
174,대신석유(주)진양주유소,서울 송파구 올림픽로 499 (풍납동),2178,송파구,2178,1965
199,평화주유소,서울 양천구 안양천로 657 (신정동),2197,양천구,2197,1902


In [28]:
df_min

,상호,주소,가격,구,max,min
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,강동구,2298,1969
14,대성산업㈜청량리주유소,서울 동대문구 왕산로 289 (청량리동),1935,동대문구,2190,1935
15,대성산업(주)동마장주유소,서울 동대문구 고산자로 405 (용두동),1935,동대문구,2190,1935
16,풍한주유소,서울 동대문구 안암로 168,1935,동대문구,2190,1935
17,동서울고속주유소,서울 동대문구 한천로 414,1935,동대문구,2190,1935
33,창원이엔지주유소,서울 동작구 사당로 59 (상도동),1964,동작구,2135,1964
43,청원주유소,서울 마포구 월드컵북로 113 (성산동),1968,마포구,2298,1968
44,성산대교셀프주유소,서울 마포구 성산로 144,1968,마포구,2298,1968
55,(주)창원이엔지 홍제지점,서울 서대문구 통일로 455 (홍제동),1960,서대문구,2059,1960
56,현대오일뱅크(주)직영 연세주유소,서울 서대문구 연희로 184,1960,서대문구,2059,1960


In [33]:
df_final = pd.merge(df_max,df_min, how='outer')

In [34]:
df_final

,상호,주소,가격,구,max,min
0,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,강동구,2298,1969
1,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,동대문구,2190,1935
2,살피재주유소,서울 동작구 상도로 334,2135,동작구,2135,1964
3,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,마포구,2298,1968
4,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,서대문구,2059,1960
...,...,...,...,...,...,...
63,(주)헨지스에너지 구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1945,도봉구,2037,1945
64,도봉제일주유소(주)송만에너지,서울 도봉구 도봉로 783 (도봉동),1945,도봉구,2037,1945
65,(주)자연에너지 햇살주유소,서울 도봉구 방학로 142 (방학동),1945,도봉구,2037,1945
66,한이에너지(주) 쌍문주유소,서울 도봉구 도봉로 547 (쌍문동),1945,도봉구,2037,1945


In [38]:
import requests
from urllib.parse import quote

In [35]:
with open('roadapikey.txt') as file:
    road_key = file.read()
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()

In [39]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f"keyword={quote('광성주유소')}&resultType=json"
url = f'{road_url}?{params1}&{params2}'
url

'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do?confmKey=devU01TX0FVVEgyMDIyMDQxMzEwMTExMDExMjQ1NzQ=&currentPage=1&countPerPage=10&keyword=%EA%B4%91%EC%84%B1%EC%A3%BC%EC%9C%A0%EC%86%8C&resultType=json'

In [41]:
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [43]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울 강동구 올림픽로 673 (천호동)'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%98%AC%EB%A6%BC%ED%94%BD%EB%A1%9C%20673%20%28%EC%B2%9C%ED%98%B8%EB%8F%99%29'

In [44]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 강동구 천호동 354-4',
    'b_code': '1174010900',
    'h_code': '1174061000',
    'main_address_no': '354',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '강동구',
    'region_3depth_h_name': '천호2동',
    'region_3depth_name': '천호동',
    'sub_address_no': '4',
    'x': '127.124241977598',
    'y': '37.5415768035116'},
   'address_name': '서울 강동구 올림픽로 673',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 강동구 올림픽로 673',
    'building_name': '광성주유소',
    'main_building_no': '673',
    'region_1depth_name': '서울',
    'region_2depth_name': '강동구',
    'region_3depth_name': '천호동',
    'road_name': '올림픽로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.124241977598',
    'y': '37.5415768035116',
    'zone_no': '05246'},
   'x': '127.124241977598',
   'y': '37.5415768035116'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [45]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])

In [46]:
lng_list, lat_list = [],[]
for i in df_final.index:
    url = f'{local_url}?query={quote(df_final.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

In [48]:
df_final['위도']= lat_list
df_final['경도']= lng_list

df_final

,상호,주소,가격,구,max,min,위도,경도
0,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,강동구,2298,1969,37.541577,127.124242
1,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,동대문구,2190,1935,37.588425,127.057395
2,살피재주유소,서울 동작구 상도로 334,2135,동작구,2135,1964,37.498360,126.951389
3,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,마포구,2298,1968,37.550595,126.955637
4,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,서대문구,2059,1960,37.595442,126.948477
...,...,...,...,...,...,...,...,...
63,(주)헨지스에너지 구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1945,도봉구,2037,1945,37.667452,127.043563
64,도봉제일주유소(주)송만에너지,서울 도봉구 도봉로 783 (도봉동),1945,도봉구,2037,1945,37.674474,127.044067
65,(주)자연에너지 햇살주유소,서울 도봉구 방학로 142 (방학동),1945,도봉구,2037,1945,37.663105,127.036402
66,한이에너지(주) 쌍문주유소,서울 도봉구 도봉로 547 (쌍문동),1945,도봉구,2037,1945,37.654005,127.037538


In [53]:
map = folium.Map(location=[df_final.위도.mean(),df_final.경도.mean()], zoom_start=12)
for i in df_final.index:
    folium.Marker(
        location=[df_final.위도[i],df_final.경도[i]],
        popup=folium.Popup(df_final.주소[i], max_width=200),
        tooltip=df_final.상호[i]
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">가격이 싼 주유소,비싼주유소 표시</h3>' 
map.get_root().html.add_child(folium.Element(title_html))
map